# Gradientマシンの起動用ノートブック

このノートブックは、Paperspace Gradientのマシンをセットアップして起動するためのスクリプトです。
起動後、リモートカーネル（Jupyter Server）として接続するためのURLも出力します。


## 1. 必要なライブラリのインストール
環境変数の読み込みのための `python-dotenv` と、Gradient操作用の `gradient` パッケージをインストールします。

In [ ]:
!pip install -U python-dotenv gradient

## 2. APIキーの取得と設定 (`.env` の読み込み)

認証にはPaperspaceのAPIキーが必要です。
プロジェクトルート(`c:\Work\pallas-ceres`)またはノートブックと同じ階層に `.env` ファイルを作成し、以下のようにAPIキーを記述してください：

```
PAPERSPACE_API_KEY=あなたのAPIキー
```

以下のセルで `.env` を読み込み、APIキーを設定します。

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

# .envファイルの読み込み（カレントディレクトリと一つ上の階層の両方を探す）
env_path = Path('.env')
if not env_path.exists():
    env_path = Path('../.env')

print(f"🔍 .envファイルを探しています: {env_path.absolute()}")
load_dotenv(dotenv_path=env_path)

API_KEY = os.getenv("PAPERSPACE_API_KEY")
if not API_KEY:
    raise ValueError(f"PAPERSPACE_API_KEY が {env_path.absolute()} に設定されていません。ファイルが存在し、中身が正しいか確認してください。")

# APIキーの適用
os.environ["PAPERSPACE_API_KEY"] = API_KEY
print("✅ APIキーを環境変数から読み込みました。")

## 3. インストールの確認
Gradient CLI が正しくインストールされ、バージョンが表示されるか確認します。

In [ ]:
!gradient version

## 4. Gradientマシンの起動と準備完了待機
起動したいマシンID（例：`ma1234abcd`）を指定してマシンを起動します。
（自分用のマシンIDはPaperspaceコンソールにて確認してください）

In [ ]:
import time
import subprocess
import json

MACHINE_ID = "YOUR_MACHINE_ID_HERE" # ← ★ ここにマシンIDを設定してください

if MACHINE_ID == "YOUR_MACHINE_ID_HERE":
    print("⚠️ MACHINE_ID を設定してください。")
else:
    print(f"🚀 マシン {MACHINE_ID} を起動しています...")
    # CLI経由でマシン起動
    subprocess.run(["gradient", "machines", "start", "--id", MACHINE_ID])
    
    print("⏳ マシンが起動して準備できるまで待機します...")
    
    # 状態が ready になるまでポーリング（最大起動待ち想定時間）
    max_retries = 30
    sleep_time = 10 
    machine_ready = False
    
    for i in range(max_retries):
        # 全マシンのステータスを取得
        result = subprocess.run(["gradient", "machines", "list"], capture_output=True, text=True)
        if MACHINE_ID in result.stdout:
            # 簡易的に state が一覧に含まれているか判定
            lines = result.stdout.split("\n")
            for line in lines:
                if MACHINE_ID in line:
                    if "ready" in line.lower() or "running" in line.lower():
                        machine_ready = True
                        break
            if machine_ready:
                break
        
        print(f"  [{i+1}/{max_retries}] 起動中... ({sleep_time}秒待機)")
        time.sleep(sleep_time)
    
    if machine_ready:
        print("\n✅ マシンの起動が完了し、準備が整いました！")
    else:
        print("\n⚠️ マシンの起動確認がタイムアウトしました。Paperspaceコンソールから状態を確認してください。")

## 5. Remote Kernel（Jupyter Server）用 URL の生成
Gradient が提供するマシンの Remote Kernel URL は、通常以下のような形式になります：
`https://<マシンID>.gradient.paperspace.com`

VSCode 等の「既存の Jupyter Server」に接続する際はこのURLを使用します。

In [ ]:
if MACHINE_ID != "YOUR_MACHINE_ID_HERE":
    remote_kernel_url = f"https://{MACHINE_ID}.paperspace.gradient.com"
    # Paperspaceの場合、ドメインが .paperspace.gradient.com または .gradient.paperspace.com のケースがありますが、
    # カスタム接続の場合は「https://{MACHINE_ID}.paperspace.gradient.com」が一般的です。
    # もしアクセスできない場合は paperspace.com の管理コンソール内「Access remote kernel」アイコンのリンクを確認してください。

    print("="*60)
    print("🔗 Remote Kernel (Jupyter Server) URL:")
    print(f"   {remote_kernel_url}")
    print("="*60)
    print("VSCode等の「Jupyter Server の選択」で上記URLを入力して接続してください。")